# Overview

At a very high level, Howso Engine is about: 

- Making an accurate prediction (even with limited or sparse data!) 

- Explaining the prediction process 

- Showing key properties of the data 

In this notebook, we will be using the adult data set as an example to demonstrate some of Howso Engine’s capabilities, including cases and features which contribute to predictions, anomalies analysis, and potential improvements to the data to gain more insight into the data.  


In [25]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from howso import engine
from howso.utilities import infer_feature_attributes
from howso.visuals import plot_feature_importances, plot_anomalies, plot_dataset

In [2]:
# Load adult data
df = pd.read_csv('data/adult.data', header=None)

# Specify column names
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 
              'education-num', 'marital-status', 'occupation',
              'relationship', 'race', 'sex', 'capital-gain', 
              'capital-loss', 'hours-per-week', 'native-country', 'target']

# Sample the data for demo purpose
df = df.sample(1_000).reset_index(drop=True)

df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target
0,27,Private,284196,10th,6,Divorced,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K
1,35,Private,150125,Assoc-voc,11,Divorced,Other-service,Unmarried,Black,Female,0,0,40,United-States,<=50K
2,20,Private,157541,Some-college,10,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K
3,35,Self-emp-not-inc,102471,HS-grad,9,Divorced,Farming-fishing,Not-in-family,White,Male,0,0,80,Puerto-Rico,<=50K
4,25,Self-emp-not-inc,108001,9th,5,Never-married,Craft-repair,Not-in-family,White,Male,0,0,15,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,29,?,188675,Some-college,10,Divorced,?,Own-child,Black,Male,0,0,40,United-States,<=50K
996,53,Local-gov,287192,1st-4th,2,Married-civ-spouse,Other-service,Husband,White,Male,0,0,32,Mexico,<=50K
997,39,State-gov,152023,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K
998,52,Private,208302,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,36,United-States,<=50K


In [3]:
partial_features = {
    "education": {"type": "nominal"}
}

# Infer features types
features = infer_feature_attributes(df, features=partial_features)

# Specify the context and action feature
action_features = ['target']
context_features = features.get_names(without=['target'])

In [4]:
# Create the trainee with custom name
t = engine.Trainee(name='Engine - Predictions and Explanations Recipe', features=features, overwrite_existing=True)

# Train
t.train(df)

# Analyze the model
t.analyze(action_features=action_features)


In [5]:
t.react_into_trainee(residuals=True)

accuracy = t.get_prediction_stats(stats=['accuracy'])['target'][0]

print("Test set prediction accuracy: {acc}".format(acc=accuracy))

Test set prediction accuracy: 0.799


# Explain

How was the predictions made? 

Howso Engine provides detailed explanation for complete model transparency. Let's examine a subset of the explanations.


## Feature importance (global)

The feature importance information provides insight into the feature[s] which were primary drivers for each of the prediction. This is important to understand in the context of AI bias and discrimination (ex. Sensitive attribute being the primary contribution to a prediction). 

This information is available at the global level (overall model), but can also be extracted at the local level (regional model for each case).

In [6]:
# Extract the global MDA (mean decrease in accuracy)
t.react_into_trainee(action_feature=action_features[0], mda_robust=True, residuals=True)
global_mda = t.get_prediction_stats(action_feature=action_features[0], stats=['mda'])
plot_feature_importances(global_mda, title="Global Mean Decrease in Accuracy (MDA)", yaxis_title="MDA")

## Feature uncertainty (global)

Are there any noisy features? 

Howso Engine’s performance is robust against noisy feature[s], and can maintain a high level of accuracy despite noisy data. 

Part of the reason  Howso Engine can maintain the level of performance despite noisy data is through characterization of feature uncertainties (residuals). The feature residuals can be extracted for user review. Note, the residuals are in the same units as the original features which makes it easy to interpret. For example, the residual for the “age” feature has the unit of years as in the original data.

Feature residuals are available at the global level (overall model) and at the local level (regional model for each case).


In [7]:
# Global feature residuals
global_feature_residuals = t.get_prediction_stats(stats=['mae']).T.rename(columns={'mae':'residuals'}).sort_values('residuals', ascending=False)
global_feature_residuals.iloc[0:10]

,residuals
fnlwgt,103630.613082
capital-gain,1778.883629
hours-per-week,9.971435
age,9.288927
occupation,0.769955
workclass,0.375080
relationship,0.348255
marital-status,0.291084
education,0.253008
education-num,0.253008


# "Show me..."

Howso Engine can be used to show interesting information pertaining to the data and model, such as anomalous cases and potential model improvements. 
 
For each prediction, Howso Engine can also extract the influential cases and boundary cases to provide an exact explanation to the prediction process. More details on what’s available can be found in the notebook “2-interpretability.ipynb”.


## Anomalous cases

Anomalous cases can exist in the data as either an outlier or inlier. Outliers are cases which are very different than other cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 



In [8]:
# Store the familiarity conviction, this will be used to identify anomalous cases
t.analyze()
t.react_into_features(familiarity_conviction_addition=True, distance_contribution=True)
stored_convictions = t.get_cases(session=t.active_session, features=df.columns.tolist() + ['familiarity_conviction_addition','.session_training_index', '.session', 'distance_contribution'])

stored_convictions

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution
0,27,Private,284196,10th,6,Divorced,Other-service,Not-in-family,White,Female,0,0,40,United-States,<=50K,13.075084,0,36928b23-3855-42d6-abc1-9c449c4bca6f,3.405597
1,35,Private,150125,Assoc-voc,11,Divorced,Other-service,Unmarried,Black,Female,0,0,40,United-States,<=50K,11.687488,1,36928b23-3855-42d6-abc1-9c449c4bca6f,4.391143
2,20,Private,157541,Some-college,10,Never-married,Craft-repair,Own-child,White,Male,0,0,40,United-States,<=50K,0.654311,2,36928b23-3855-42d6-abc1-9c449c4bca6f,1.895770
3,35,Self-emp-not-inc,102471,HS-grad,9,Divorced,Farming-fishing,Not-in-family,White,Male,0,0,80,Puerto-Rico,<=50K,0.822998,3,36928b23-3855-42d6-abc1-9c449c4bca6f,6.355230
4,25,Self-emp-not-inc,108001,9th,5,Never-married,Craft-repair,Not-in-family,White,Male,0,0,15,United-States,<=50K,7.256599,4,36928b23-3855-42d6-abc1-9c449c4bca6f,4.572374
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,29,?,188675,Some-college,10,Divorced,?,Own-child,Black,Male,0,0,40,United-States,<=50K,11.135537,995,36928b23-3855-42d6-abc1-9c449c4bca6f,4.409850
996,53,Local-gov,287192,1st-4th,2,Married-civ-spouse,Other-service,Husband,White,Male,0,0,32,Mexico,<=50K,2.229436,996,36928b23-3855-42d6-abc1-9c449c4bca6f,5.247838
997,39,State-gov,152023,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,45.614267,997,36928b23-3855-42d6-abc1-9c449c4bca6f,3.557260
998,52,Private,208302,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,0,0,36,United-States,<=50K,0.433202,998,36928b23-3855-42d6-abc1-9c449c4bca6f,1.574636


In [9]:
# Threshold to determine which cases will be deemed anomalous
convict_threshold = 0.75

# Extract the anomalous cases
low_convicts = stored_convictions[stored_convictions['familiarity_conviction_addition'] <= convict_threshold ].sort_values('familiarity_conviction_addition', ascending=True)

# Average distance contribution will be used to determine if a case is an outlier or inlier
average_dist_contribution = low_convicts['distance_contribution'].mean()

# A case with distance contribution greater than average will be tagged as outlier, and vise versa for inliers
cat = ['inlier' if d < average_dist_contribution else 'outlier' for d in low_convicts['distance_contribution']]
low_convicts['category'] = cat

## Outliers

Let’s examine a few outlier cases. Outliers are cases which are very different than other cases.

In [10]:
# Extract the outliers cases
outliers = low_convicts[low_convicts['category'] == 'outlier'].reset_index(drop=True)
outliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,46,Local-gov,222115,Bachelors,13,Divorced,Adm-clerical,Not-in-family,White,Female,99999,0,40,United-States,>50K,0.063868,531,36928b23-3855-42d6-abc1-9c449c4bca6f,16.625742,outlier
1,39,Self-emp-inc,114844,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,65,United-States,>50K,0.081267,170,36928b23-3855-42d6-abc1-9c449c4bca6f,13.687340,outlier
2,74,Private,188709,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,50,United-States,>50K,0.086136,58,36928b23-3855-42d6-abc1-9c449c4bca6f,12.713613,outlier
3,50,Private,183173,HS-grad,9,Married-civ-spouse,Prof-specialty,Husband,White,Male,99999,0,40,United-States,>50K,0.086615,872,36928b23-3855-42d6-abc1-9c449c4bca6f,12.538070,outlier
4,55,Private,197399,Masters,14,Married-civ-spouse,Exec-managerial,Husband,White,Male,99999,0,55,United-States,>50K,0.087340,67,36928b23-3855-42d6-abc1-9c449c4bca6f,12.306585,outlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,24,State-gov,27939,Some-college,10,Married-civ-spouse,Other-service,Husband,Asian-Pac-Islander,Male,0,0,24,?,<=50K,0.695685,106,36928b23-3855-42d6-abc1-9c449c4bca6f,6.612450,outlier
59,60,Private,139586,Assoc-voc,11,Widowed,Exec-managerial,Unmarried,Asian-Pac-Islander,Female,0,0,40,United-States,>50K,0.705769,806,36928b23-3855-42d6-abc1-9c449c4bca6f,6.595706,outlier
60,28,Private,330466,Bachelors,13,Never-married,Prof-specialty,Not-in-family,Asian-Pac-Islander,Male,0,0,40,Hong,<=50K,0.716683,242,36928b23-3855-42d6-abc1-9c449c4bca6f,6.567220,outlier
61,43,Private,972354,Some-college,10,Married-civ-spouse,Other-service,Husband,White,Male,0,0,48,United-States,<=50K,0.727575,535,36928b23-3855-42d6-abc1-9c449c4bca6f,6.546316,outlier


In [11]:
# Cache global non-robust residuals into trainee
t.react_into_trainee(residuals=True)

# Get the case_feature_residual_convictions, influential_cases and boundary_cases
details = {'robust_computation': True,
           'global_case_feature_residual_convictions': True, 
           'local_case_feature_residual_convictions': True}

# Specify outlier cases
outliers_indices = outliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=outliers_indices, 
                  preserve_feature_values=df.columns.tolist(), 
                  leave_case_out=True, 
                  details=details)

In [12]:
# Extract the global and local case feature residual convictions
global_case_feature_residual_convictions = pd.DataFrame(results['explanation']['global_case_feature_residual_convictions'])[df.columns.tolist()]
local_case_feature_residual_convictions = pd.DataFrame(results['explanation']['local_case_feature_residual_convictions'])[df.columns.tolist()]

In [13]:
plot_anomalies(outliers, local_case_feature_residual_convictions, title="Outliers", yaxis_title="Residual Conviction")

The heat map explains the reason why each case was an outlier. The darker the shade of red, the higher the contribution to the case being an outlier. 

## Inliers

Let’s examine a few inlier cases. Inliers are cases which are too similar to other cases and do not follow the expected distribution. Inliers can be an indication of a fraudulent case that is “too good to be true”. 

In [14]:
# Get the inlier cases
inliers = low_convicts[low_convicts['category'] == 'inlier'].reset_index(drop=True)
inliers

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,category
0,53,Private,165745,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.136095,402,36928b23-3855-42d6-abc1-9c449c4bca6f,0.663712,inlier
1,54,Private,175594,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.146106,74,36928b23-3855-42d6-abc1-9c449c4bca6f,0.712693,inlier
2,31,Private,191834,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,>50K,0.156952,365,36928b23-3855-42d6-abc1-9c449c4bca6f,0.803843,inlier
3,50,Private,163671,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.160435,127,36928b23-3855-42d6-abc1-9c449c4bca6f,0.772870,inlier
4,44,Private,113324,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,<=50K,0.173865,353,36928b23-3855-42d6-abc1-9c449c4bca6f,0.835041,inlier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
151,41,Private,59938,HS-grad,9,Divorced,Adm-clerical,Unmarried,White,Female,0,0,43,United-States,<=50K,0.739122,45,36928b23-3855-42d6-abc1-9c449c4bca6f,2.026632,inlier
152,43,Private,108945,Some-college,10,Divorced,Prof-specialty,Unmarried,White,Female,0,0,38,United-States,<=50K,0.739144,603,36928b23-3855-42d6-abc1-9c449c4bca6f,2.005334,inlier
153,26,Private,259505,Some-college,10,Never-married,Other-service,Own-child,White,Male,0,0,40,United-States,<=50K,0.741642,479,36928b23-3855-42d6-abc1-9c449c4bca6f,1.991535,inlier
154,28,Private,217200,HS-grad,9,Never-married,Machine-op-inspct,Not-in-family,White,Male,0,0,50,United-States,<=50K,0.747674,57,36928b23-3855-42d6-abc1-9c449c4bca6f,2.001021,inlier


In [15]:
# Specify the inlier cases
inliers_indices = inliers[['.session', '.session_training_index']].values

# React to get the details of each case
results = t.react(case_indices=inliers_indices, 
                  preserve_feature_values=df.columns.tolist(), 
                  leave_case_out=True, 
                  details=details)

In [16]:
# Extract the global and local case feature residual convictions
global_case_feature_residual_convictions = pd.DataFrame(results['explanation']['global_case_feature_residual_convictions'])[df.columns.tolist()]
local_case_feature_residual_convictions = pd.DataFrame(results['explanation']['local_case_feature_residual_convictions'])[df.columns.tolist()]

In [17]:
plot_anomalies(inliers, local_case_feature_residual_convictions, title="Inliers", yaxis_title="Residual Conviction")

The heat map explains the reason why each case was an inlier. The darker the shade of blue, the higher the contribution the to case being an inlier.

## Potential improvements

Sparse regions of the model or under defined problems can make it difficult to make an accurate prediction. Howso Engine can be used to identify potential data, or model improvements by examining the residual conviction and density.

In [18]:
# Identify cases for investigation
partial_train_df = stored_convictions
partial_train_cases = partial_train_df[['.session', '.session_training_index']]


In [19]:
# Residual convictions are output via the local_case_feature_residual_convictions explanation
details = {'global_case_feature_residual_convictions':True}

# Get the residual convictions for the specified cases
new_result = t.react(case_indices=partial_train_cases.values.tolist(), 
                     leave_case_out=True, 
                     preserve_feature_values=df.drop(action_features, axis=1).columns.tolist(), 
                     action_features=action_features,
                     details=details)

In [20]:
# Extract residual conviction
target_residual_convictions = [ x['target'] for x in new_result['explanation']['global_case_feature_residual_convictions'] ]

# Binarize residual conviction
convict_threshold = 0.75
low_residual_conviction = [1 if x <= convict_threshold else 0 for x in target_residual_convictions]

# Density is just the inverse of distance_contribution
density = 1 / partial_train_df['distance_contribution']

# Add new features to the dataframe
partial_train_df['density'] = density
partial_train_df['target_residual_conviction'] = target_residual_convictions
partial_train_df['low_residual_conviction'] = low_residual_conviction

In [21]:
partial_train_df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,...,hours-per-week,native-country,target,familiarity_conviction_addition,.session_training_index,.session,distance_contribution,density,target_residual_conviction,low_residual_conviction
0,27,Private,284196,10th,6,Divorced,Other-service,Not-in-family,White,Female,...,40,United-States,<=50K,13.075084,0,36928b23-3855-42d6-abc1-9c449c4bca6f,3.405597,0.293634,2.082822e+02,0
1,35,Private,150125,Assoc-voc,11,Divorced,Other-service,Unmarried,Black,Female,...,40,United-States,<=50K,11.687488,1,36928b23-3855-42d6-abc1-9c449c4bca6f,4.391143,0.227731,4.789228e-01,1
2,20,Private,157541,Some-college,10,Never-married,Craft-repair,Own-child,White,Male,...,40,United-States,<=50K,0.654311,2,36928b23-3855-42d6-abc1-9c449c4bca6f,1.895770,0.527490,3.352711e+12,0
3,35,Self-emp-not-inc,102471,HS-grad,9,Divorced,Farming-fishing,Not-in-family,White,Male,...,80,Puerto-Rico,<=50K,0.822998,3,36928b23-3855-42d6-abc1-9c449c4bca6f,6.355230,0.157351,8.878975e+09,0
4,25,Self-emp-not-inc,108001,9th,5,Never-married,Craft-repair,Not-in-family,White,Male,...,15,United-States,<=50K,7.256599,4,36928b23-3855-42d6-abc1-9c449c4bca6f,4.572374,0.218705,5.922678e-01,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,29,?,188675,Some-college,10,Divorced,?,Own-child,Black,Male,...,40,United-States,<=50K,11.135537,995,36928b23-3855-42d6-abc1-9c449c4bca6f,4.409850,0.226765,2.082822e+02,0
996,53,Local-gov,287192,1st-4th,2,Married-civ-spouse,Other-service,Husband,White,Male,...,32,Mexico,<=50K,2.229436,996,36928b23-3855-42d6-abc1-9c449c4bca6f,5.247838,0.190555,1.236138e+00,0
997,39,State-gov,152023,11th,7,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,<=50K,45.614267,997,36928b23-3855-42d6-abc1-9c449c4bca6f,3.557260,0.281115,2.082822e+02,0
998,52,Private,208302,HS-grad,9,Married-civ-spouse,Sales,Husband,White,Male,...,36,United-States,<=50K,0.433202,998,36928b23-3855-42d6-abc1-9c449c4bca6f,1.574636,0.635067,5.905324e-01,1


In [28]:
# Helper function to resize the data points
def get_sizes(min_size, max_size, series):
    min_value = series.min()
    max_value = series.max()
    
    m = (max_size - min_size) / (max_value - min_value)
    
    sizes = series * m + min_size
    return (sizes)

partial_train_df["density"] = get_sizes(5, 500, partial_train_df["density"])

In [29]:
plot_dataset(partial_train_df, x="age", y="education-num", size="density", hue="low_residual_conviction", alpha=0.4)

The above graph is a visualization of the data set in 2-dimensions, with the color as an indication of residual conviction and the size representing the density of the data. More specifically, the orange color represents the low conviction points (points which are very uncertain), and small size represents low density. Therefore, adding more data to the region with small, orange points can improve model performance. 


On the other hand, an orange point that is large would be an indication that this case lies in an dense region but was not predictable. Hence, this will be an indication where the problem is not well defined, or the data is missing key features.  
